In [1]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from os import listdir
import re
import math
from sklearn.metrics import confusion_matrix

In [2]:
FILEPATH = 'ece661_pics\\hw8_image\\'

In [3]:
class Pattern():
    ''' 
        Class for handling pattern.
    '''
    P = 8
    ARRAY2VALUE = np.array([2**(7-i) for i in range(P)])

    def __init__(self, pattern):
        self.pattern = pattern 
        self.find_min_val()
    
    def __repr__(self):
        return  (f'Pattern = {self.pattern}\n'
                f'Min pattern = {self.min_pattarn}\n'
                f'Min Value = {self.min_value}\n')
    
    def find_min_val(self):
        ''' 
            Find the min value pattern.
        '''
        min_value = 2**self.P
        pattern = self.pattern.tolist()
        for i in range(self.P):
            pattern.append(pattern.pop(0))
            value = np.dot(self.ARRAY2VALUE, np.array(pattern))
            if value < min_value:
                self.min_value = value
                self.min_pattarn = pattern.copy()
                min_value = value

    def encode(self):
        '''
            Encode the pattern.
        '''
        if self.min_value == 0:
            return 0
        elif self.min_value == 2**self.P - 1:
            return self.P
        
        runs = 0
        ones = 0
        prev = self.min_pattarn[0]
        if prev == 1:
            ones += 1
            runs *= 1
        for i in range(1, self.P):
            current = self.min_pattarn[i]
            if prev != current:
                runs +=1 
            if runs > 2:
                return self.P + 1
            if current == 1:
                ones += 1
            prev = current
        return ones


In [4]:
class Image():
    ''' 
        Class for storing images.
    '''
    
    def __init__(self, name):
        self.name = name    
        self.image = None  
        self.hist = None

    def load(self):
        self.image = cv.imread(self.name)
        self.image_gray = cv.cvtColor(self.image, cv.COLOR_BGR2GRAY)

    def show(self):
        if self.image is None:
            self.load()
        plt.imshow(cv.cvtColor(self.image, cv.COLOR_BGR2RGB))
    
    def extract_pattern(self, p=8, r=1):
        '''
            Extract histrogram pattern.
        '''
        if self.image is None:
            self.load()
        
        if self.hist is not None:
            return self.hist
        
        img = self.image_gray
        h, w = img.shape
        du = math.cos(2*math.pi/p)
        dv = math.sin(2*math.pi/p)
        # Compute linear interpolation
        # 1. Turn 3 x 3 to 9 x 1
        # 2. Dot product with 8 x 9 matrix below
        interpolation = np.array([
            [0, 0, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, (1-du)*(1-dv), (1-du)*(dv), 0, (du)*(1-dv), (dv)*(dv)],
            [0, 0, 0, 0, 0, 1, 0, 0, 0],
            [0, (du)*(1-dv), (dv)*(dv), 0, (1-du)*(1-dv), (1-du)*(dv), 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0],
            [(dv)*(dv), (du)*(1-dv), 0, (1-du)*(dv), (1-du)*(1-dv), 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, (1-du)*(dv), (1-du)*(1-dv), 0, (dv)*(dv), (du)*(1-dv), 0],
        ])

        patterns = []
        for i in range(1, w-1):
            for j in range(1, h-1):
                # Extract 3 x 3 kernal
                kernal = img[j-1:j+2, i-1:i+2]
                # Apply linear interpolation
                pattern = np.dot(interpolation, kernal.ravel().T)
                # Compare to center pixel
                pattern = pattern > kernal[1][1]
                pattern = Pattern(pattern.astype(np.uint8))
                # Use encode function of Pattern class define above
                patterns.append(pattern.encode())

        self.hist, self.bin_edges = np.histogram(patterns, bins=[i for i in range(p+3)], density=True)
        return self.hist

    def plot_histogram(self, savename=None):
        if self.hist is None:
            self.extract_pattern() 
        
        plt.bar(self.bin_edges[0:-1], self.hist)
        if savename is not None:
            savename = f'{FILEPATH}{savename}.png'
            plt.savefig(savename)


In [13]:
def load_image(img_classes):
    '''
        Load images from database.
    '''

    trainning_set = {}
    testing_set = {}
    
    for img_class in img_classes:
        trainning_set[img_class] = []
        testing_set[img_class] = []
        # Load training sets
        directory = f'{FILEPATH}training'
        imgs = listdir(directory)
        imgs_in_class = [re.findall(f'{img_class}.*', img) for img in imgs]
        for img in imgs_in_class:
            if img:
                full_path = f'{directory}\\{img[0]}'
                trainning_set[img_class].append(Image(full_path))

        # Load testing set
        directory = f'{FILEPATH}testing'
        imgs = listdir(directory)
        imgs_in_class = [re.findall(f'{img_class}.*', img) for img in imgs]
        for img in imgs_in_class:
            if img:
                full_path = f'{directory}\\{img[0]}'
                testing_set[img_class].append(Image(full_path))

    return trainning_set, testing_set

In [14]:
img_classes = ['rain', 'cloudy', 'shine', 'sunrise']
[imgs_train, imgs_test] = load_image(img_classes)

In [44]:
result = []
truth = []
for class_idx_test, img_class_test in enumerate(img_classes):
    for img_test in imgs_test[img_class_test]:
        print(img_test.name) # Image under test
        distances = [] # Distance to each train image
        indiecs = [] # Class index of each train image
        for class_idx_train, img_class_train in enumerate(img_classes):
            for img_train in imgs_train[img_class_train]:
                distance = np.linalg.norm(img_train.extract_pattern() - img_test.extract_pattern())
                distances.append(distance)
                indiecs.append(class_idx_train)
        distances = np.array(distances)
        indiecs = np.array(indiecs)
        min_distances = np.sort(distances) # Sorting distance array
        min_distances_idx = np.where(distances < min_distances[5]) # Find 5 least distance
        # Get Class index of most common class of 5 least distance
        predicted = np.bincount(indiecs[min_distances_idx]).argmax()  
        print(f'Ground Truth class {img_classes[class_idx_test]}. Predicted class {img_classes[predicted]}\n')
        truth.append(class_idx_test)
        result.append(predicted)

confusion_matrix(truth, result)

ece661_pics\hw8_image\testing\rain206.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain207.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain208.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain209.jpg
Ground Truth class rain. Predicted class cloudy

ece661_pics\hw8_image\testing\rain210.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain211.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain212.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain213.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain214.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\rain215.jpg
Ground Truth class rain. Predicted class rain

ece661_pics\hw8_image\testing\cloudy291.jpg
Ground Truth class cloudy. Predicted class cloudy

ece661_pics\h

array([[9, 1, 0, 0],
       [0, 8, 2, 0],
       [0, 3, 5, 2],
       [0, 5, 1, 4]], dtype=int64)

In [43]:
total = len(imgs_train['sunrise'])
for i, img in enumerate(imgs_train['sunrise']):
    print(f'{i}/{total}')
    img.extract_pattern()

0/347
1/347
2/347
3/347
4/347
5/347
6/347
7/347
8/347
9/347
10/347
11/347
12/347
13/347
14/347
15/347
16/347
17/347
18/347
19/347
20/347
21/347
22/347
23/347
24/347
25/347
26/347
27/347
28/347
29/347
30/347
31/347
32/347
33/347
34/347
35/347
36/347
37/347
38/347
39/347
40/347
41/347
42/347
43/347
44/347
45/347
46/347
47/347
48/347
49/347
50/347
51/347
52/347
53/347
54/347
55/347
56/347
57/347
58/347
59/347
60/347
61/347
62/347
63/347
64/347
65/347
66/347
67/347
68/347
69/347
70/347
71/347
72/347
73/347
74/347
75/347
76/347
77/347
78/347
79/347
80/347
81/347
82/347
83/347
84/347
85/347
86/347
87/347
88/347
89/347
90/347
91/347
92/347
93/347
94/347
95/347
96/347
97/347
98/347
99/347
100/347
101/347
102/347
103/347
104/347
105/347
106/347
107/347
108/347
109/347
110/347
111/347
112/347
113/347
114/347
115/347
116/347
117/347
118/347
119/347
120/347
121/347
122/347
123/347
124/347
125/347
126/347
127/347
128/347
129/347
130/347
131/347
132/347
133/347
134/347
135/347
136/347
137/347
138/34